# Extracting SkySim5000 DM halo
Authors : Michel Aguena

(based on [this](https://github.com/LSSTDESC/CLCosmo_Sim/blob/main/notebooks/SkySim5000_firstcheck.ipynb) in the [ClCosmo_Sim](https://github.com/LSSTDESC/CLCosmo_Sim/) repo from C. Payerne & C. Combet)

In [ ]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import scipy.integrate
import astropy.units as u
from astropy.table import Table
import GCRCatalogs

%matplotlib inline

In [ ]:
skysim_cat = GCRCatalogs.load_catalog('skysim5000_v1.1.1_small')

In [ ]:
cosmo_ss  = skysim_cat.cosmology

In [ ]:
cosmo_ss

## Extract DM haloes from the catalog in a given mass and redshift range. 

In [ ]:
%%time
# get list of halos in a given redshift and mass range
dm_halos = Table(skysim_cat.get_quantities(
    ['halo_id', 'halo_mass', 'redshift','ra', 'dec',
     'baseDC2/sod_halo_mass','baseDC2/sod_halo_radius'],
    filters=[
        f'halo_mass > {10**12.8}',
        'is_central==True',
        f'redshift>{0}',
        f'redshift<{3}']
))
dm_halos['m200c'] = dm_halos['baseDC2/sod_halo_mass']/cosmo_ss.h

In [ ]:
print(f'There are {len(dm_halos):,} halos in this mass (Mfof) and redshift range')

In [ ]:
plt.scatter(dm_halos['ra'], dm_halos['dec'], marker='.', s=0.001)
plt.xlabel('ra [deg]')
plt.ylabel('dec [deg]')

In [ ]:
plt.scatter(dm_halos['m200c'], dm_halos['halo_mass'], marker='.', s=0.1)
plt.xlabel('$M_{200c}$ [$M_\odot$]')
plt.ylabel('FoF mass [$M_\odot$]')
diag = np.min([*dm_halos['m200c'], *dm_halos['halo_mass']]), np.max([*dm_halos['m200c'], *dm_halos['halo_mass']])
plt.plot(diag, diag, c='r', ls='--')
plt.axvline(1e13, c='g')
plt.xscale('log')
plt.yscale('log')

We see that all halos with $M_{200c}>10^{13}M_\odot$ are included!

## Define a redshift and mass range for the comparison of data and prediction, and filter the data accordingly


In [ ]:
zbins = np.linspace(0, 3, 31)
mbins = 10**np.array([13.0, 13.5, 13.8, 14.2, 15])

In [ ]:
nc_meas = np.histogram2d(dm_halos['m200c'], dm_halos['redshift'], bins=(mbins, zbins))[0]

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, figsize=(16, 8))

for i, ax in enumerate(axes.flatten()):
    ax.hist(zbins[:-1], bins=zbins, weights=nc_meas[i],
           label=f'[{np.log10(mbins[i]):.1f}:{np.log10(mbins[i+1]):.1f}]')
    ax.legend(handlelength=0, handletextpad=0, loc=1)
    
for ax in axes[-1,:]:
    ax.set_xlabel('redshift')
    
for ax in axes[:,0]:
    ax.set_ylabel('number counts')

## Prediction using CCL

$$
n(M_\alpha, z_i)
=\int_{z_i}^{z_{i+1}} dV(z)
\int_{M_\alpha}^{M_{\alpha+1}} dM \frac{dn(M, z)}{dM}
$$

In [ ]:
# Define CCL Cosmology from SkySim cosmology
import pyccl as ccl
cosmo = ccl.Cosmology(Omega_c=cosmo_ss.Om0-cosmo_ss.Ob0, Omega_b=cosmo_ss.Ob0,
                      h=cosmo_ss.h, sigma8=cosmo_ss.sigma8, n_s=cosmo_ss.n_s, Neff=3.04)

print(cosmo)

### Differential comoving volume

In [ ]:
def dV_over_dOmega_dz(z):
    a = 1./(1. + z)
    da = ccl.background.angular_diameter_distance(cosmo, a) 
    E = ccl.background.h_over_h0(cosmo, a)
    return ((1.+z)**2)*(da**2)*ccl.physical_constants.CLIGHT_HMPC/cosmo['h']/E 

### CCL mass functions
**Tinker et al. 2008** & **Bocquet et al. 2016**:

In [ ]:
hmd_200c = ccl.halos.MassDef(200, 'critical')
def tinker08(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncTinker08(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M
def bocquet16(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncBocquet16(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

### Integral

In [ ]:
def nc_theo(mfunc, logmmin, logmmax, zmin, zmax):
    return scipy.integrate.dblquad(
        lambda logm, z: mfunc(logm, z)*dV_over_dOmega_dz(z),
        zmin, zmax, lambda x:logmmin, lambda x:logmmax,
        epsabs=1.e-4, epsrel=1.e-4)[0]
    print(out)
    return out

Solid Angle:

In [ ]:
DeltaOmega = 50 * np.pi**2/180**2

Compute integrals

In [ ]:
%%time
tinker08_nc = [
    [nc_theo(tinker08, np.log10(mmin), np.log10(mmax), zmin, zmax)*DeltaOmega
        for zmin, zmax in zip(zbins, zbins[1:])]
    for mmin, mmax in zip(mbins, mbins[1:])]

In [ ]:
%%time
bocquet16_nc = [
    [nc_theo(bocquet16, np.log10(mmin), np.log10(mmax), zmin, zmax)*DeltaOmega
        for zmin, zmax in zip(zbins, zbins[1:])]
    for mmin, mmax in zip(mbins, mbins[1:])]

Compare results:

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, figsize=(16, 8))

for i, ax in enumerate(axes.flatten()):
    ax.hist(zbins[:-1], bins=zbins, weights=nc_meas[i],
           label=f'[{np.log10(mbins[i]):.1f}:{np.log10(mbins[i+1]):.1f}]')
    legend = ax.legend(handlelength=0, handletextpad=0)
    ax.plot(zbins[:-1]+.05, np.array(tinker08_nc[i]), label='Tk08')
    ax.plot(zbins[:-1]+.05, np.array(bocquet16_nc[i]), label='Bq16')
    if i==0:
        ax.legend(ax.lines, [l._label for l in ax.lines], loc=2)
        ax.add_artist(legend)
    
for ax in axes[-1,:]:
    ax.set_xlabel('redshift')
    
for ax in axes[:,0]:
    ax.set_ylabel('number counts')